In [19]:
from collections import deque
import string

In [20]:
def get_stacks(stacks, stack_row):
    ''' Takes the stacks dictionary and the current stack row, 
        return the stacks dict with the stack rows info added '''
    
    # use this to keep track of the stack we are working with
    stack_counter = 1

    # loop stack row to extract the crates
    for i in range(1, len(stack_row), 4):

        # get the crate label
        crate_label = stack_row[i]

        # skip over the empty rows
        if crate_label != ' ':

            # put the crate label in the appropriate place in the stack
            current_stack = stacks.get(stack_counter, deque())
            current_stack.append(crate_label)
            stacks[stack_counter] = current_stack

        # move to the next stack in the row
        stack_counter += 1

    return stacks


def extract_stack_info(data):
    ''' Take the raw input data, returns the stack info'''

    # keep track of the stacks in a dict so they can be indexed
    stacks = {}

    upper_case_set = set(string.ascii_uppercase)

    # loop over the stack row in the data
    for stack_row in data:
        row_set = set(stack_row)

        if not stack_row or not (upper_case_set & row_set):
            # starting from the top if the row is empty, or if it has number then
            # we are no longer in the stack information part of the input
            break
        else:
            # get the info from that row of the stack
            stacks = get_stacks(stacks, stack_row)

    # reverse it so it is in the right order, we are reading from the top of the stack down
    for key in stacks.keys():
        stacks[key].reverse()

    return stacks


def get_instructions(data):
    ''' Take the data set, return the instructions'''

    # instruction list, need to presever order
    instructions = []

    for line in data:

        # all the instruction rows start with the word 'move' so we use this test
        if 'move' not in line:
            continue

        else:
            # we care about how many crate to move (quantity), from where (origin), to where (destination)
            # the instructions are in a predictable order so we hard code that 
            instruction_pieces = line.split(' ')
            instructions.append({'quantity': int(instruction_pieces[1]),\
                                 'origin': int(instruction_pieces[3]),\
                                 'destination': int(instruction_pieces[5])})

    return instructions

def execute_instructions(stacks, instructions, version='9000'):
    ''' Takes the stacks, instructions and crane version, returns the stacks after instructions have been executed'''

    # execute the instruction
    for instruction in instructions:

        # check to see what crane we are using
        if version == '9000':
            # here we grab crate one at a time
            for _ in range(instruction['quantity']):

                # remove the crate from its origin stack
                crate = stacks[instruction['origin']].pop()

                # add the crate to its destination stack
                stacks[instruction['destination']].append(crate)

        elif version == '9001':
            # here we move crates all at once from the origin stack
            crates = [stacks[instruction['origin']].pop() for _ in range(instruction['quantity'])]

            # make sure they are in the right order
            crates.reverse()

            # put the crates in their destination stack
            stacks[instruction['destination']] = stacks[instruction['destination']] + deque(crates)

        else:
            # sometimes we get the crane version number wrong
            Exception('Wrong crane version number')
    
    return stacks


def get_stack_tops(stacks):
    ''' Takes the stacks, return the crate on top of each'''

    stack_tops = ''
    stacks_index = range(1, len(stacks)+1)
    for index in stacks_index:
        stack_tops += stacks[index][-1]
    
    return stack_tops
    

In [21]:
def help_the_elfs(input_path, crane_version='9000'):
    ''' Takes the location of the raw data and crane version,
        return the updated stacks '''

    # read in the raw data
    with open(input_path, 'r') as f:
        data = f.read().splitlines()

    # extract the stack information and instructions
    stacks = extract_stack_info(data)
    instructions = get_instructions(data)

    # execute the instructions on the stack
    reordered_stacks = execute_instructions(stacks, instructions, crane_version)

    # find the top crates for each stack
    stack_tops = get_stack_tops(reordered_stacks)
    
    return stack_tops

In [22]:
def test_part_1():
    ''' Returns True if part 1 works against the test data '''

    input_path = "test_input.txt"
    test_solution = 'CMZ'
    stack_tops = help_the_elfs(input_path, crane_version='9000')

    return stack_tops==test_solution

def test_part_2():
    ''' Return True if part 2 works against the test data'''
    input_path = "test_input.txt"
    test_solution = 'MCD'
    stack_tops = help_the_elfs(input_path, crane_version='9001')
    return stack_tops==test_solution

In [23]:
if test_part_1():
    input_path = "input.txt"
    answer = help_the_elfs(input_path, crane_version='9000')
    msg = 'The tops of the stacks with crane 9000 are {answer}'
    print(msg.format(answer=answer))

The tops of the stacks with crane 9000 are QPJPLMNNR


In [24]:
if test_part_2():
    input_path = "input.txt"
    answer = help_the_elfs(input_path, crane_version='9001')

    msg = 'The tops of the stacks with crane 9001 are {answer}'
    print(msg.format(answer=answer))


The tops of the stacks with crane 9001 are BQDNWJPVJ
